# Optimization and Analytics
### Bachelor in Data Science and Engineering 
### First homework: Linear and Discrete Models (Topics 1,2)


Dimitar Ilev

# Problem formulation
- The *Drives A/S* is a company spceilized in producing different types of drives for variable speed motor control or power conversion applications. They have a wide range of Variable Frequency Drives(VFD) to offer.
- The firm has been producing for around 10 years now. Thus it has a collection of reliable data to determine the expected amount of devices necessary to produce of each type.
- The company wants to optimize its profits and for that reason they are in desperate need of a reliable solution to find out how many units of which Unit to produce, in order to achieve the best profitability.

## Goals/ Objectives

- Maximize the profits
- Stay inside given limits
  - Budget
  - Demand
  - Labour time

In [23]:
# IMPORTS
from pyomo.opt import SolverFactory
import sys
from pyomo.environ import *
import pyomo.environ as pyo
import pandas as pd

# Data
- For the purpose of training this model, I decided to use a smaller sample size, so the more focus can be put on the functionallity of the model.

In [63]:
#mydata = pd.read_csv("C:\\BSc in EIB\\5. Semester\\Repos\Repo - Optimization and Analytics\\dt1.csv",)
mydata = pd.read_csv('dt1.csv')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_seq_items', None)
mydata

C:\Users\Dimi\AppData\Local\Temp\ipykernel_14380\3494933311.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,VFD_Type_Nr.,Sell_Price,Material_Cost_EUR,Net_Profit_EUR,Labour_Time_Unit_minutes,Total_Demand_Units
0,1785,60,24,36,19,73
1,1328,80,23,57,17,47
2,1927,128,18,110,10,48
3,1719,137,21,116,11,70
4,1849,108,23,85,20,77
5,1149,92,15,77,15,68
6,1484,119,17,102,19,54
7,1565,79,22,57,19,60
8,1435,104,24,80,13,48
9,1868,145,19,126,20,51


# Parameter lists

In [25]:
# Create python lists for each category
type_number_array = []
for i in mydata.iloc[:,0]:
  type_number_array.append(i)
  
sell_price_array = []
for i in mydata.iloc[:,1]:
  sell_price_array.append(float(i))

material_cost_array = []
for i in mydata.iloc[:,2]:
  material_cost_array.append(float(i))

net_profit_array = []
for i in mydata.iloc[:,3]:
  net_profit_array.append(float(i))

labour_hours_array = []
for i in mydata.iloc[:,4]:
  labour_hours_array.append(float(i))

total_demand_array = []
for i in mydata.iloc[:,5]:
  total_demand_array.append(i)

In [26]:
L = range(len(total_demand_array))
L

range(0, 22)

## Variables
- The company has a total range of 21 different types of VDF.
- They are all customizable and can be each produced in a timewindow of 10-25mins.

In [27]:
# Create model object
model = ConcreteModel('Profit Maximisation')

# Each Variable -> Number of each VFD
model.x = Var([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21], domain=Binary, doc = 'Number of Units produced')


## Constraints
- The company has a production plan for the year. With an estimated maximum demand for the units.
- The maximum amount of labour hours is 360.000min = 6000 hours per year. Which is 30.000min = 500h per month.
- The budget of the company is roughly 10.000EUR per month.
- Each unit has to be produced at least once.

In [59]:
# Labour hours
# #Type1 * LH_Type1 + #Type2 * LH_Type2 + ... <= 30.000 minutes
for i in L:
   model.cons1 = Constraint(expr = model.x[i+1]*labour_hours_array[i]<= 30000)

# Material costs
# #Type1*Mc_Type1 + #Type2 * Mc_Type2 + ... <= 10.000 EUR
   model.cons2 = Constraint(expr = model.x[i+1]*material_cost_array[i]<= 10000)

# Total demand
# Each VFD has to stay under its max demand
   model.cons3 = Constraint(expr = model.x[i+1] <= total_demand_array[i])

# Each Type has to be produced at least once
  #Type >= 0

    'pyomo.core.base.constraint.ScalarConstraint'>) on block Profit
    Maximisation with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.ScalarConstraint'>) on block Profit
    Maximisation with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.ScalarConstraint'>) on block Profit
    Maximisation with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.ScalarConstraint'>) on block 

- Tried to define the constraints in the method listed below, but that still did not work properly!

In [61]:
# L = L[:-1] # Cut length so no index out of bounds
# def Labour_time_cons(model, i):
#   return sum(model.x[i+1]*labour_time_array[i]  for i in L)<= 60000
# model.Labour_time_cons = Constraint(rule = Labour_time_cons, doc ='Maximum number of Labour time')

# def Budget_cons(model,i):
#   return sum(model.x[i+1]*material_cost_array[i] for i in L)<= 100000
# model.Budget_cons = Constraint(model.x, rule = Budget_cons, doc ='Maximum budget')

# def Total_demand_cons(model, i):
#   return sum(model.x[i+1] <= total_demand_array[i] for i in L)
# #model.Total_demand_cons = Constraint(model.x, rule = Total_demand_cons, doc =' Total demand')

# Objective and Solver

In [57]:
# Objective function
# P = #Type1 * NetProfitType1 + #Type2 * NetProfitType2 + ...

#model.objective = Objective(expr = sum(model.x[i+1]*net_profit_array[i] for i in L),sense=maximize,doc='Max Profit')

#model.OBJ = Objective(rule=sum(model.x[i+1]*net_profit_array[i] for i in L),sense=maximize,doc='Max Profit')

def objective_rule(model):
  return sum(model.x[i+1]*net_profit_array[i] for i in L)
model.objective = Objective(rule=objective_rule, sense=maximize, doc='Profit Maximisation')

# Solver object
solvername='glpk'
solverpath_folder='C:\\w64'
solverpath_exe='C:\\w64\\glpsol'

sys.path.append(solverpath_folder)
solver=SolverFactory(solvername, executable=solverpath_exe)

# solve
results = solver.solve(model)
results.write()
if results.solver.status:
    model.pprint()

#display solution
print('\nProfit = ', model.objective())

print('\nDecision Variables')
for i in L:
  print('x = ', model.x[i+1])


print('\nConstraints')
print('Labour time in min  = ', model.cons1())
print('Material costs = ', model.cons2())
print('Total demand = ', model.cons3())

    'pyomo.core.base.objective.ScalarObjective'>) on block Profit Maximisation
    with a new Component (type=<class
    'pyomo.core.base.objective.ScalarObjective'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_component().
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1113.0
  Upper bound: 1113.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 14
  Number of nonzeros: 17
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Terminati

In [53]:
model.objective()

1113.0

# Conclusion
The solver executes only one element from the list and thus the result is only covering the beginning of the process. I tried to find a way to fix that but my attempts did not work properly and it messed up the code even more. I suppose the solution would be to iterate over the the list, but I did not see why this does not work on the first iteration.

For a demand of only one unit the time results in 16min and with a total material cost of 21EUR.Expected was that the Optimizer shows a much greater result in the total price outcome and a higher time investment. I suppose the iteration over the list does not work properly yet and it couldt give out a realistic result.

Based on the listed information the forecast should have been:
- Total Labour time: 25.000min/month -> ~ 400h/month
- Material costs: ~ 9.000EUR/month
- Total demand: ~ 1.200Units/Month
